# RAG асистент

## Устанавливаем зависимости

In [1]:
%pip install -qq langchain langchain-community langchain-qdrant unstructured yandexcloud requests markdown langchain-ollama

Note: you may need to restart the kernel to use updated packages.


## Скачиваем документ

In [2]:
#import requests
#file_link = "https://teach-in.ru/file/synopsis/pdf/introduction-to-quantum-physics-lectures-rubtsov-M.pdf"
#response = requests.get(file_link)

#if response.status_code == 200:
#    print("Загрузка прошла успешно)")
#else:
#    print(f"Ошибка [ {response.status_code} ]")

Сохраняем документ на временном диске сессии

In [3]:
# Локальное расположение документа
from pathlib import Path
file_path = Path("introduction-to-quantum-physics-lectures-rubtsov-M.pdf")

In [4]:
#with open("introduction-to-quantum-physics-lectures-rubtsov-M.pdf", "w") as file:
#    file.write(response.text)

## Pre-Retrieval

### Чанкование данных

**Импортируем библиотеки**

In [6]:
from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    Docx2txtLoader,
    UnstructuredMarkdownLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain.schema import Document

#### Загрузка документа

In [7]:
def get_loader(file_path: Path):
    suffix = file_path.suffix.lower()

    loaders = {
        ".txt": TextLoader,
        ".pdf": PyPDFLoader,
        ".docx": Docx2txtLoader,
        ".md": UnstructuredMarkdownLoader
    }

    if suffix not in loaders:
        raise ValueError(f"Неподдерживаемый формат файла: {suffix}")

    return loaders[suffix](file_path)

**Загружаем документ**

In [8]:
loader = get_loader(file_path)
document = loader.load()

Проверяем корректность загрузки

In [9]:
print(document[0].metadata)
print(document[0].page_content[:500])

{'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2020-12-22T13:00:30+00:00', 'author': '', 'keywords': '', 'moddate': '2020-12-22T20:59:17+03:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'introduction-to-quantum-physics-lectures-rubtsov-M.pdf', 'total_pages': 211, 'page': 0, 'page_label': '1'}
МЕХАНИКА  • СЛЕПКОВ АЛЕКСАНДР ИВАНОВИЧ
КОНСПЕКТ ПОДГОТОВЛЕН СТУДЕНТАМИ, НЕ ПРОХОДИЛ  
ПРОФ. РЕДАКТУРУ И МОЖЕТ СОДЕРЖАТЬ ОШИБКИ.  
СЛЕДИТЕ ЗА ОБНОВЛЕНИЯМИ НА VK.COM/TEACHINMSU.
ВВЕДЕНИЕ В 
КВАНТОВУЮ ФИЗИКУ 
РУБЦОВ
АЛЕКСЕЙ НИКОЛАЕВМЧ
ФИЗФАК МГУ
КОНСПЕКТ ПОДГОТОВЛЕН 
СТУДЕНТАМИ, НЕ ПРОХОДИЛ 
ПРОФ. РЕДАКТУРУ И МОЖЕТ 
СОДЕРЖАТЬ ОШИБКИ.  
СЛЕДИТЕ ЗА ОБНОВЛЕНИЯМИ 
НА VK.COM/TEACHINMSU .
ЕСЛИ ВЫ ОБНАРУЖИЛИ 
ОШИБКИ ИЛИ ОПЕЧАТКИ, 
ТО СООБЩИТЕ ОБ ЭТОМ, 
НАПИСАВ СООБЩЕСТВУ 
VK.COM/TEACHINMSU .
ФИЗИЧЕСКИЙ  



#### Чанкование

**Конфигурационные параметры**

* `chunk_size` - максимальный размер чанка, кол-во символов
* `chunk_overlap` - размер перекрытия чанков (нахлёста), кол-во символов

In [10]:
chunk_size = 500
chunk_overlap = 150

**Настройка нарезчика данных на чанки**

Будем использовать нарезчик, учитывающий иерархическую структуру документа, т.е. абзаци, главы и т.д.

In [11]:
recursive_character_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

**Чанкование**

In [12]:
chunks = recursive_character_splitter.split_documents(document)

Проверка корректности чанкования

In [13]:
starting_chunk = 4 # Начальный чанк
count_chunks = 2 # Количество выводимых чанков

for i, chunk in enumerate(chunks[starting_chunk:starting_chunk + count_chunks]):
    print(f'---[ {i + starting_chunk} ]---\n {chunk.page_content}')

---[ 4 ]---
 1.4 РаботыЭйнштейна.Фотоэффект.Люминесценция.Законыфотоэффекта 11
1.5 Ультрафиолетовая катастрофа . . . . . . . . . . . . . . . . . . . . . . . . 14
1.6 Неравенства Белла . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
1.7 Соотношение классической и квантовой физики . . . . . . . . . . . . . . 16
1.8 Атомная система единиц. Фундаментальные константы . . . . . . . . . 17
2 Лекция 2. Корпускулярные и волновые свойства света 19
---[ 5 ]---
 1.8 Атомная система единиц. Фундаментальные константы . . . . . . . . . 17
2 Лекция 2. Корпускулярные и волновые свойства света 19
2.1 История физики. Энергия и импульс релятивистских частиц. Эффект
Комптона . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 19
2.2 Волновые свойства света. Опыт Юнга-Френеля . . . . . . . . . . . . . . 23
2.3 Опыт Винера. Стоячие волны . . . . . . . . . . . . . . . . . . . . . . . . 24


## Retrieval

**Импорт библиотек**

In [14]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant.qdrant import QdrantVectorStore

#from langchain_community.embeddings import YandexGPTEmbeddings
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings


### Развёртывание векторной БД

**Конфигурационные параметры**

* `collection_name` - наименование коллекции в БД, в которой будут храниться загруженные данные.
* `vector_size` - количество значений в векторном представлении, его длина. У используемых эмбеддинговых моделей от Яндекса максимальная длина 256, поэтому здесь также её придерживаемся.
* `distance` - метрика для определения расстояния между векторами. Лучше представлять, как обыкновенное расстояние между точками в пространстве.

In [15]:
collection_name = "physics_collection"
vector_size = 1024     # Размер векторов, возможно стоит прейти на более маленькую размерность - 256     
distance = 'Cosine'

Загружаем переменные окружения из файла .env

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()

QDRANT_URL = os.getenv("QDRANT_URL")
#QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
#YANDEX_API_KEY = os.getenv("YANDEX_API_KEY")
#YANDEX_FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")

**Подключению к векторной БД**

In [18]:
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    #api_key=QDRANT_API_KEY,  # Если есть токен, например в  Qdrant Cloud
)

**Создание коллекции данных в кластере**

Создание коллекции

In [19]:
if collection_name not in [collection.name for collection in qdrant_client.get_collections().collections]:
    qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vector_size,
        distance=Distance(distance)
    )
)

**Создание векторного хранилища**

Для создания эмбедингов будем использовать модель YandexGPTEmbeddings

In [ ]:
# Загрузка модели для создания эмбеддингов через YandexGPT
#yandex_embedding = YandexGPTEmbeddings(
#    api_key=YANDEX_API_KEY,
#    folder_id=YANDEX_FOLDER_ID
#)

# Загрузка модели для создания эмбеддингов через  Ollama
local_embedding = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://127.0.0.1:11434" 
)


In [21]:
# Инициализация векторного хранилища
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    #embedding=yandex_embedding
    embedding=local_embedding
)

### Добавление базы знаний в векторную БД

#### Принцип работы модели для создания эмбеддингов

Искомый текст:

In [31]:
text = chunks[100].page_content  # Текст из 100 чанка страницы
print(text)

атомной физики, для описания каких-то субатомных по шкале масштабов процессов
естественно необходимо ввести какую-то систему единиц, в которой этой неприятно-
сти не будет. Иногда постоянную Планка пишут вместо¯h2 ¯h´2 или наоборот ¯h это
10´27, поэтому ¯h2 вместо ¯h´2 это 104 порядка лишних, физикам сразу хочется, что-
то сделать с лишним порядком. Поэтому если считать в атомной системе единиц, то
конечно хотя бы таких казусов не будет.


Текст преобразованный в эмбеддинг (векторное представление):

In [55]:
result = local_embedding.embed_query(text)
print(result[:10])  # Выводим первые 10 элементов 

[-0.008481814, 0.016389186, -0.009079622, 0.029147211, -0.025374198, 0.01527201, -0.00016501374, -0.0014643716, 0.024831874, 0.02323621]


**Добавляем чанки в векторную БД**

In [56]:
result = vector_store.add_documents(chunks) 
print(result[:10])  # Выводим первые 10 элементов 

['7cef2c0318dd41fabc8cae115b34d5e2', 'f02393fa1f524b839789e0e20aa62993', '6de605647d3f45928d08dc0bcca8aec9', '6791430660a741258e828dfe61eb2f08', 'bd25e3083d7342f8b16f5af796801e45', '1289833f90e2488581d3a584c15be7e6', 'b642fee4c3f540418207296b32d670e8', 'bd10c745e9164bcaa5faf4bbf300f9c2', '697ed49d0bdb4d5d8a91123fecf9509d', '1db3355b60e24340b7488854d5cb74c5']


#### Поиск в векторной БД

**Конфигурационные параметры**

* `k` - максимальное количетсво возвращаемых валидных чанков. Другими словами - это топ `k` самых близких к запросу чанков.
* `query` - пользовательский запрос.

In [37]:
k = 5
#query = "Перескажи в кратце о чём говорится в предисловии к изданию."
query = "Кратко перескажи о чём говорится в Лекции 3."


**Поиск валидных чанков**

In [38]:
searching_chunks = vector_store.similarity_search(
    query=query,
    k=k
)

Топ `k` наиболее близких к запросу чанков

In [39]:
for i, searching_chunk in enumerate(searching_chunks):
    print(f'{i + 1}. {searching_chunk.page_content}')

1. 3 Лекция 3. Корпускулярно-волновой дуализм. Строение атома 36
3.1 Модель Томсона . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 36
3.2 Описание Бора . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 38
3.3 Описание Де-Бройля . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 40
4 Лекция 4. Частица в потенциальной яме 51
4.1 Уравнение Шредингера . . . . . . . . . . . . . . . . . . . . . . . . . . . . 51
2. 3 Лекция 3. Корпускулярно-волновой дуализм. Строение атома 36
3.1 Модель Томсона . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 36
3.2 Описание Бора . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 38
3.3 Описание Де-Бройля . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 40
4 Лекция 4. Частица в потенциальной яме 51
4.1 Уравнение Шредингера . . . . . . . . . . . . . . . . . . . . . . . . . . . . 51
3. 3 Лекция 3. Корпускулярно-волновой дуализм. Строение атома 36
3.1 Модель Томсона . . . . . . . . . 

## Generative

**Импорт библиотек**

In [40]:
from langchain_core.prompts import ChatPromptTemplate
#from langchain_community.llms import YandexGPT
from langchain_community.llms.ollama import Ollama

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

**Конфигурационные параметры**
* `model_name` - наименование модели
* `temperature` - температура - отвечает за степень галлюционирования модели, т.е. добавления в ответы несвязных данных, выбросов и т.д. Данный параметр примимает занчения в промежутке `[0, 1]`
* `max_tokens` - максимальное количество токенов, доступных для полного цикла работы модели, т.е. сумма колличества токенов из входного запроса плюс - в ответе. Обыно у моделей данный параметр может принимать различные максимумы, поэтому необходимо читать документацию конкретного экземпляра.

In [41]:
#model_name = 'yandexgpt'
model = "gemma3:12b-it-q4_K_M"
temperature = 0.7
base_url="http://127.0.0.1:11434"
#max_tokens = 8000


**Загрузка LLM**



In [43]:
OllamaGPT = Ollama(
    #api_key=YANDEX_API_KEY,
    #folder_id=YANDEX_FOLDER_ID,
    base_url=base_url,
    model=model,
    temperature=temperature,
    #max_tokens=max_tokens
)

**Создание шаблона запроса**

In [44]:
system_prompt ="""
**Роль**
Ты — Физик с большим стажем, который пишет объяснения студентам и не любит долгих разговоров.

**Инструкции**
1. Используй приведённый контекст для ответа на вопрос.
2. Если ты не можешь найти ответ в контексте, так и скажи: 'В документе отсутствуют данные для формирования ответа.', не пытайся придумать ответ.
3. Твои высказывания должны быть связными по смыслу и чётко доносить мысль из контекста.
4. Сфомируй ответ простым текстом, не более 10-20 слов.

**Контекст**
{context}
"""

In [45]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        ('human', '{input}')
    ]
)

**Финализация RAG pipeline`а**

In [46]:
question_answer_chain = create_stuff_documents_chain(
    OllamaGPT,
    prompt
)

chain = create_retrieval_chain(
    vector_store.as_retriever(),
    question_answer_chain
)

Отправляем запрос:

In [47]:
print(query)

Кратко перескажи о чём говорится в Лекции 3.


In [48]:
answer = chain.invoke({"input": query})

In [49]:
print(answer["answer"])

Лекция 3 посвящена корпускулярно-волновому дуализму и строению атома, включая модели Томсона, Бора и Де-Бройля.


### Общение с RAG-ассистентом

In [50]:
def get_answer(query):
    k = 20

    searching_chunks = vector_store.similarity_search(
        query=query,
        k=k
    )

    system_prompt ="""
        **Роль**
        Ты — DevOps с большим стажем, который недавно написал документацию и не любит долгих разговоров.

        **Инструкции**
        1. Используй приведённый контекст для ответа на вопрос.
        2. Если ты не можешь найти ответ в контексте, так и скажи: 'В документе отсутствуют данные для формирования ответа.', не пытайся придумать ответ.
        3. Твои высказывания должны быть связными по смыслу и чётко доносить мысль из контекста.

        **Контекст**
        {context}
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', system_prompt),
            ('human', '{input}')
        ]
    )

    question_answer_chain = create_stuff_documents_chain(
        OllamaGPT,
        prompt
    )

    chain = create_retrieval_chain(
        vector_store.as_retriever(),
        question_answer_chain
    )

    answer = chain.invoke({"input": query})

    return answer["answer"]

**Примеры**

In [51]:
query = "В чём цель данной книги?"
print(get_answer(query))

Цель книги - объяснить разницу между пояснением и формулой при решении задач о движении двух связанных частиц. Ответ известен из классической механики и не меняется в квантовом случае.


In [52]:
query = "Для кого написана данная книга?"
print(get_answer(query))

Книга написана для студентов, изучающих "Введение в квантовую физику" у Рубцова Алексея Николаевича. Конспект подготовлен студентами и не проходил профессиональную редактуру. Следите за обновлениями на VK.COM/TEACHINMSU.



In [53]:
query = "Какие были предпосылки к появлению DevOps, как профессии?"
print(get_answer(query))

В документе отсутствуют данные для формирования ответа.


In [54]:
query = "Что отражает волновая функция?"
print(get_answer(query))

Волновая функция — это функция координат и времени. Физическим смыслом обладают не все её составляющие, общая фаза волновой функции не отражает наблюдаемые величины и не имеет прямого физического смысла.
